# Wikipedia Retriever

In [11]:
from langchain_community.retrievers import WikipediaRetriever

In [12]:
# Initialize the retriever (optimal: set language and top_k)
retriever = WikipediaRetriever(top_k_results=2, lang='en')

In [23]:
# Define your query
query = 'the geopolitical history of india and pakistan'

# Get relevant Wikipedia documents
docs = retriever.invoke(query)

In [24]:
docs

[Document(metadata={'title': 'Indo-Pakistani wars and conflicts', 'summary': 'Since the partition of British India in 1947 and subsequent creation of the dominions of India and Pakistan, the two countries have been involved in a number of wars, conflicts, and military standoffs. A long-running dispute over Kashmir and cross-border terrorism have been the predominant cause of conflict between the two states, with the exception of the Indo-Pakistani War of 1971, which occurred as a direct result of hostilities stemming from the Bangladesh Liberation War in erstwhile East Pakistan (now Bangladesh).', 'source': 'https://en.wikipedia.org/wiki/Indo-Pakistani_wars_and_conflicts'}, page_content='Since the partition of British India in 1947 and subsequent creation of the dominions of India and Pakistan, the two countries have been involved in a number of wars, conflicts, and military standoffs. A long-running dispute over Kashmir and cross-border terrorism have been the predominant cause of con

In [ ]:
for i, doc in enumerate(docs):
    print(f'\n--- Result {i+1} ---')
    print(f'Content:\n{doc.page_content}') # truncate for display


--- Result 1 ---
Content:
Since the partition of British India in 1947 and subsequent creation of the dominions of India and Pakistan, the two countries have been involved in a number of wars, conflicts, and military standoffs. A long-running dispute over Kashmir and cross-border terrorism have been the predominant cause of conflict between the two states, with the exception of the Indo-Pakistani War of 1971, which occurred as a direct result of hostilities stemming from the Bangladesh Liberation War in erstwhile East Pakistan (now Bangladesh).


== Background ==

The Partition of India came in 1947 with the sudden grant of independence. It was the intention of those who wished for a Muslim state to have a clean partition between independent and equal "Pakistan" and "Hindustan" once independence came.
Nearly one third of the Muslim population of India remained in the new India.
Inter-communal violence between Hindus, Sikhs and Muslims resulted in between 200,000 and 2 million casualti

# Vector Store Retriever

In [1]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [7]:
# Step 2: Initialize embedding model
embedding_model = OpenAIEmbeddings()

In [8]:
# Step 3: Create Chroma vector store in memory
vectorstore = Chroma.from_documents(
    documents= documents,
    embedding=embedding_model,
    collection_name='my_colllection'
)

In [9]:
# Step 4: Convert vectorstore into a retriever
retriever = vectorstore.as_retriever(search_kwargs={'k':2})

In [10]:
query = 'What is Chroma used for?'
results = retriever.invoke(query)

In [11]:
for i, doc in enumerate(results):
    print(f'\n--- Result {i+1} ---')
    print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
LangChain helps developers build LLM applications easily.


In [12]:
results = vectorstore.similarity_search(query, k=2)
for i, doc in enumerate(results):
    print(f'\n--- Result {i+1} ---')
    print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
LangChain helps developers build LLM applications easily.


# Maximal Marginal Relevance (MMR) -

In [21]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [22]:
from langchain_community.vectorstores import FAISS

In [23]:
# Step 1: Initialize OpenAI embeddings
embedding_model = OpenAIEmbeddings()

In [24]:
# Step 2: Create the FAISS vector store from documents
vectorstore = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [25]:
# Enable MMR in retriever
retriever = vectorstore.as_retriever(
    search_type='mmr', # This enables mmr.
    search_kwargs={'k':3, 'lambda_mult': 1} # k = top results, lambda_mult = relevance-diversity balance
)

In [26]:
query = 'What is langchain?'
results = retriever.invoke(query)

In [27]:
for i, doc in enumerate(results):
    print(f'\n--- Result {i+1} ---')
    print(doc.page_content)


--- Result 1 ---
LangChain is used to build LLM based applications.

--- Result 2 ---
LangChain makes it easy to work with LLMs.

--- Result 3 ---
LangChain supports Chroma, FAISS, Pinecone, and more.


As we can see we used lambda_mult = 1 (most relevant), therefore the first 2 results are very similar(king of same)

In [28]:
retriever = vectorstore.as_retriever(
    search_type='mmr', # This enables mmr.
    search_kwargs={'k':3, 'lambda_mult': 0} # k = top results, lambda_mult = relevance-diversity balance
)

In [29]:
query = 'What is langchain?'
results = retriever.invoke(query)

In [30]:
for i, doc in enumerate(results):
    print(f'\n--- Result {i+1} ---')
    print(doc.page_content)


--- Result 1 ---
LangChain is used to build LLM based applications.

--- Result 2 ---
Embeddings are vector representations of text.

--- Result 3 ---
MMR helps you get diverse results when doing similarity search.


As we can see using lambda_mult ve most different results.

In [31]:
retriever = vectorstore.as_retriever(
    search_type='mmr', # This enables mmr.
    search_kwargs={'k':3, 'lambda_mult': 0.5} # k = top results, lambda_mult = relevance-diversity balance
)

In [32]:
query = 'What is langchain?'
results = retriever.invoke(query)

In [33]:
for i, doc in enumerate(results):
    print(f'\n--- Result {i+1} ---')
    print(doc.page_content)


--- Result 1 ---
LangChain is used to build LLM based applications.

--- Result 2 ---
Embeddings are vector representations of text.

--- Result 3 ---
LangChain supports Chroma, FAISS, Pinecone, and more.


As we can see using lambda_mult = 0.5 not only gave relevant but different results as well.

# Multi Query Retriever

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
# from langchain_google_genai import ChatGoogleGenerativeAI - the reason of not using it is that ChatOpenAI works well with OpenAIEmbeddings.
from langchain.retrievers import MultiQueryRetriever

In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [14]:
# Initalize OpenAI embeddings
embedding_model = OpenAIEmbeddings()

In [15]:
# Create FIASS vector store
vectorstore = FAISS.from_documents(
    documents=all_docs,
    embedding=embedding_model
)

In [16]:
# Create retrievers
similarity_retriever = vectorstore.as_retriever(search_type='similarity', search_kwargs={'k':5})

In [17]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={'k':5}),
    llm=ChatOpenAI(model='gpt-4.1-nano')
)

In [18]:
# Query
query = 'How to improve energy levels and maintain balance?'

In [19]:
# Retreved documents
similarity_results = similarity_retriever.invoke(query)
multiquery_results = multiquery_retriever.invoke(query)

In [20]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print('-'*160)

for i, doc in enumerate(multiquery_results):
    print(f'\n--- Result {i+1} ---')
    print(doc.page_content)


--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 3 ---
Regular walking boosts heart health and can reduce symptoms of depression.

--- Result 4 ---
Deep sleep is crucial for cellular repair and emotional regulation.

--- Result 5 ---
The solar energy system in modern homes helps balance electricity demand.
----------------------------------------------------------------------------------------------------------------------------------------------------------------

--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 3 ---
Deep sleep is crucial for cellular repair and emotional regulation.

--- Result 4 ---
Regular walking boosts heart health and can reduce symptoms of dep

# Contextual Compression Retriever

In [32]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document

In [33]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [34]:
# Create a FAISS vector store from the docuemnts
embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding_model)

base_retriever = vectorstore.as_retriever(search_kwargs={'k':2})

In [35]:
# Set up compressor using an LLM
llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash')
compressor = LLMChainExtractor.from_llm(llm)

In [36]:
# Create a contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [37]:
# Query the retriever
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [38]:
for i, doc in enumerate(compressed_results):
    print(f'\n--- Result {i+1} ---')
    print(doc.page_content)


--- Result 1 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 2 ---
The chlorophyll in plant cells captures sunlight during photosynthesis.


Some more Retrievers are -
1. ParentDocumentRetriever
2. TimeWeightedVectorRetriever
3. EnsembleRetriever
4. ArxivRetriever
5. MultiVectorRetriever
6. BM25Retriever
7. SelfQueryRetriever

They can be furthur explored on - https://python.langchain.com/docs/integrations/retrievers/